## Step 1: Library import

In [19]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.model_selection import GridSearchCV

import xray_data #y data: 1 = NORMAL, 0 = PNEUMONIA


## Step: Load and preprocess data
Note: data is resized and preprocessed as read, as a memory optimization.

In [2]:
#Constants
scale = 200
label_filter = ['NORMAL','PNEUMONIA']

In [3]:
X_dev_orig, y_dev_orig = xray_data.load_dev(scale,label_filter)
print(f'X_dev_orig, y_dev_orig shape: {X_dev_orig.shape, y_dev_orig.shape}')
print(f'y_dev_orig shape for NORMAL cases: {y_dev_orig[y_dev_orig ==1].shape}')
print('----')

X_dev_orig, y_dev_orig shape: ((16, 40000), (16,))
y_dev_orig shape for NORMAL cases: (8,)
----


In [4]:
# cut of for training samples of each class, only 230 normal rows
test_cutoff = 230

X_test, y_test = xray_data.load_test(scale,label_filter,test_cutoff)
print(f'X_test, y_test shape: {X_test.shape, y_test.shape}')
print(f'y_test shape for NORMAL cases: {y_test[y_test ==1].shape}')
print('----')

X_test, y_test shape: ((460, 40000), (460,))
y_test shape for NORMAL cases: (230,)
----


In [5]:
# cut of for training samples of each class, only 1300 normal rows
train_cutoff = 1300

X_train, y_train = xray_data.load_train(scale,label_filter,train_cutoff)

In [6]:
# I'm not sure what the final stratify parameter is doing.  We'll want to revist when we do full cross validation

X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size = .5, stratify = y_train )
print(f'X_train, y_train shape: {X_train.shape, y_train.shape}')
print(f'y_train shape for NORMAL cases: {y_train[y_train ==1].shape}')
print('----')
print(f'X_dev, y_dev shape: {X_dev.shape, y_dev.shape}')
print(f'y_dev shape for NORMAL cases: {y_dev[y_dev ==1].shape}')

X_train, y_train shape: ((1300, 40000), (1300,))
y_train shape for NORMAL cases: (650,)
----
X_dev, y_dev shape: ((1300, 40000), (1300,))
y_dev shape for NORMAL cases: (650,)


## Single Model training

In [21]:

base_model = svm.SVC()

base_model.fit(X_train,y_train)
y_pred = base_model.predict(X_dev)
print(f'Single Model accuracy: {accuracy_score(y_pred,y_dev)}')

Single Model accuracy: 0.9576923076923077


## Gridsearch model

In [22]:
param_grid={'C':[0.1,.5,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly','linear']}

svc = svm.SVC()
CV_model = GridSearchCV(svc,param_grid)

print(f'Best fit model parameters: {CV_model.best_params_}')
CV_model.fit(X_train,y_train)
y_pred = base_model.predict(X_dev)
print(f'CV Model accuracy: {accuracy_score(y_pred,y_dev)}')

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'